Give the average sentiment PER CANTON or per city (more precise is imposible)

In [1]:
import numpy as np
import pandas as pd

In [10]:
insta=pd.read_json("harvest3r_instagram_data_12-01_0.json");
twitter=pd.read_json("harvest3r_twitter_data_12-01_0.json");

In [48]:
twitter._source[0]

{'author_avatar_img': 'https://pbs.twimg.com/profile_images/539163907437248513/A9twlFRN_bigger.jpeg',
 'author_gender': 'UNKNOWN',
 'author_link': 'https://twitter.com/frausenn',
 'author_name': 'Frau Senn',
 'bucket': 1452619500070,
 'canonical': 'https://twitter.com/frausenn/status/686962375145164801',
 'date_found': '2016-01-12T17:26:32Z',
 'domain': 'twitter.com',
 'hashcode': 'A3JQNBSWThhoKYXCsEeX8mr21qY',
 'index_method': 'SOURCE_TASK_COMPOSITE',
 'lang': 'de',
 'links': ['https://t.co/XXHKCpM7nn'],
 'main': 'Baloo und ich sind bereit für den Feierabend. pic.twitter.com/XXHKCpM7nn',
 'main_checksum': 'BPQcuf2P8bO0Xrzd7CnCGOK_CNE',
 'main_format': 'TEXT',
 'main_length': 72,
 'permalink': 'https://twitter.com/frausenn/status/686962375145164801',
 'published': '2016-01-12T17:26:08Z',
 'resource': 'https://twitter.com/frausenn/status/686962375145164801',
 'sentiment': 'NEUTRAL',
 'sequence': 1452619592000013785,
 'sequence_range': 97670,
 'site': 'twitter.com',
 'source_content_chec

In [42]:
twitter._source[6].get('main')

'미안해요, 선약이 있어서….'

In [61]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from langdetect import detect
from nltk.stem import PorterStemmer

In [71]:
#Parsing each Tweet into word
for i in range(0,20): #len(twitter) instead of 20
    sentence=twitter.iloc[i]["_source"];
    sentence=sentence.get('main');
    
    # 1) Get Language of tweet!
    
    language=detect(sentence);
    
    if language=='en' or language=='de' or language=='it': #Add the other of language we want manually!
        
        langDict = {'en': 'english', 'de': 'german', 'it':'italian'} #Add accordingly

        # 2) Delete the stop_words (useless ones)
        stop_words = set(stopwords.words(langDict[language])) #Replace with "English", "German", "French", "Arabic" or whatever.
        word_tokens = word_tokenize(sentence)
        filtered_sentence = [w for w in word_tokens if not w in stop_words]
        filtered_sentence = []
        for w in word_tokens:
            if w not in stop_words:
                filtered_sentence.append(w);
        print(filtered_sentence);
        
        #For lexical understanding, might want to use WordNet in english, Germanet for german and MultiWordNet for Italian or Spanish
    

['Baloo', 'bereit', 'Feierabend', '.', 'pic.twitter.com/XXHKCpM7nn']
['Vierter', 'Platz', 'Wendy', 'Holdener', 'Slalom', 'Flachau', '.', 'http', ':', '//www.bote.ch', 'pic.twitter.com/1gAyQtnfrR']
['#', 'TagderBildung', 'Kanton', 'Zürich', '.', 'Da', 'freue', 'besonders', ',', '400', 'Gymnasiasten', 'Winterthur', 'aufzutreten', '...', 'pic.twitter.com/e4L3yx4Jyq']
['10', 'must-download', 'Google', 'Chrome', 'add-ons', 'surfing', 'web', 'http', ':', '//ti.me/1mQEDk9', 'vía', '@', 'Techland']
['Das', 'Internet', 'Realtime', '!', 'Lasst', 'Seite', 'mal', '10', 'Minuten', 'offen', 'schaut', '!', 'Ist', 'echt', 'spannend', ',', 'abgeht', '!', 'http', ':', '//rros.ch/1OfLSeM']
['Die', 'Strecke', 'Täsch-Zermatt', 'ab', '09.30', 'Uhr', 'wegen', 'Lawinengefahr', 'weiteres', 'gesperrt', '.']
['@', 'broennimann', '@', 'Ugugu', '@', 'metamythos', 'AZ', '``', 'getrauen', "''", 'http', ':', '//m.aargauerzeitung.ch/schweiz/roger-koeppel-verharmlost-nazi-fuehrer-hermann-goering-129979655', '…']
['Ich'

In [69]:
sentence=twitter.iloc[1]["_source"];
sentence=sentence.get('main');
    
    # 1) Get Language of tweet!
    
language=detect(sentence);
print(language)

so
